In [1]:
using ODE
using ForwardDiff # Mostly relies on package DualNumbers -> check in more detail later


# Subfunctions used for MBAM

In [ ]:
# Return f(x) given log(x)
function log_deriv_wrapper(f::Function, x::Vector; log_specific=ones(size(x)) )
    y = copy(x)
    # Assuming some of the input parameters (x) are in log space, passes the original (exponentiated) values on to the function f
    if sum(log_specific==1)==prod(size(x)) # all the parameters are in log space
        y = exp(y)
    else
        for i1 = 1:size(log_specific)[1]
            if log_specific[i1]==1
                y[i1]=exp(y[i1])
            end
        end
    end
    
    return f(y)
end

In [ ]:
# Convert a parameter vector (or specific parameters) to log values
function to_log(x::Vector; log_specific=ones(size(x)))
    y = copy(x) # To avoid passing by reference!!!
    
    if sum(log_specific==1)==prod(size(x)) # all the parameters are in log space
        y = log(y)
    else
        for i1 = 1:size(log_specific)[1]
            if log_specific[i1]==1
                y[i1]=log(y[i1])
            end
        end
    end
    
    return y
end

In [ ]:
# Given parameters x, selected parameters d and a canvas c, draw a scatter point at x[d] on the d-dim canvas c
function scatter_param(x::Vector, d, c)
    Plots.plot(x[d])
end

rcond(A::StridedMatrix) = LAPACK.gecon!('1', lufact(A).factors, norm(A, 1))

### Geodesic differential equation

As in Transtrum 2015 Supplemental

$$
\frac{d^2}{d\tau^2} \theta = [\mathbf{J}^T \mathbf{J}]^{-1} \nabla f \frac{v^T \mathbf{H}_f v}{\|v\|^2} \\
\frac{d}{d\tau} \theta = v
$$

In [42]:
# Defines the geodesic differential equation with respect to a cost function f_cost and residual function f_res
function geodesic_ode(t, y, f_cost::Function, f_res::Function)
    sz2 = round(size(y)[1]/2);
    theta = y[1:sz2]; # First half of vector is d-dim position
    v = y[(sz2+1):end]; # Second half of y vector is d-dim velocity
    
    result = HessianResult(theta);
    ForwardDiff.hessian!(result, f_cost, theta);
    
    grad = ForwardDiff.gradient(result); # Gradient in parameter space
    hess = ForwardDiff.hessian(result); # Hessian in parameter space
    
    jac = ForwardDiff.jacobian(f_res, theta);
    f_jac(theta,m) = ForwardDiff.jacobian(f_res, theta)[m,:]
    hess_m = cell(size(jac)[1],1); # Stores the second derivative of the m-th residual 
    A = zeros(size(jac)[1],1)
    for m1 = 1:(size(jac)[1])
        f_jac_m(theta) = f_jac(theta,m1);
        hess_m[m1] = ForwardDiff.jacobian(f_jac_m, theta)
        A[m1] = (v'*hess_m[m1]*v)[1]/(norm(v)^2);
    end
    
    print(size(A))
    print(size(jac))
    
    (D,V) = eig(hess)
    #print(rcond(hess))
    
    print(sort(D)[1])
    print("\n\n")
    if rcond(hess) < 1e-18
        dy = copy(y);
        dy[1:end]=0
    else   
        dy = copy(y);
        dy[1:(sz2)] = v # Change in theta is v
        dy[(sz2+1):end] = (jac'*jac) \ (jac' * A) # dy[2] is change in v (as y[2] is v), change in v is dy[1]: the second derivative
        #print(grad)
    end
    
    ## Matlab version (just take a step towards the smallest eigenvector)
    #dy[1:(sz/2)] = V[:,sortperm(D)[1]]
    #dy[(sz/2+1):end] = 0;
    return dy
end

geodesic_ode (generic function with 1 method)

# MBAM main function

In [14]:
function MBAM(f_costs::Function, x0::Vector, f_reds::Function; model_iters=0, boundary_time=10, stepsize=0.1, log_specific=ones(size(x0)), verbose=0)
    x_red = to_log(x0);
    # Initialize variables we want to remember in the end (otherwise their scope is limited to within the for cycle)
    x = x_red;
    V = x_red;
    D = x_red;
    x0 = x_red
    v0 = x_red;
    v = x_red;
    t_out = x_red;
    y_out = x_red;
    
    for m1 = model_iters # Outer loop, reduces the number of parameters in the model each step
        f_cost = x1 -> f_costs(x1, model_id=m1); # Current cost function
        g_cost = x1 -> log_deriv_wrapper(f_cost, x1); # Define which parameters we're taking in log space ([default: all])
        f_res = x1 -> f_costs(x1, model_id=m1, return_res=1)
        g_res = x1 -> log_deriv_wrapper(f_res, x1);
        
        x = x_red; # Initial point in parameter space
        H = ForwardDiff.hessian(g_cost,x)
        (D,V) = eig(H)
        v = V[:,sortperm(D)[1]]; # Initial velocity in parameter space
        print(x)
        print(v)
        
        (t_out, y_out) = ODE.ode23((t,y) -> geodesic_ode(t,y,g_cost,g_res), [x; v], [0,boundary_time])
        
        x = y_out[end][1:size(x)[1]] # Final point in parameter space (on the boundary)
        v = y_out[end][(size(x)[1]+1):end] # Final velocity in parameter space (reduceable combination)
        
        print(x)
        print(v)
        x_red = f_reds(x, model_id=m1+1, in_log=1); # Transformed representation
    end
    
    f_cost = x -> f_costs(x, model_id=model_iters[end]); # Current cost function
    g_cost = x -> log_deriv_wrapper(f_cost, x); # Define which parameters we're taking in log space ([default: all])
    #g_cost(x)
    #print((x, V, x_red, g_cost(x)))
    H = ForwardDiff.hessian(g_cost,x)
    (D,V) = eig(H)
    return x, g_cost(x), v, D, x_red, t_out, y_out
end #line 35

MBAM (generic function with 1 method)

using Plotly
using Plots


#Plots.plot([1,2,3,4,5])
scatter_param([1,2,3,4,5],[2,4],1)
scatter_param([1,2,3,4,5],[1,5],1)

In [32]:
A = [2 0;
0 2]

2x2 Array{Int64,2}:
 2  0
 0  2

In [36]:
A \ eye(2) # = A^-1 * I

2x2 Array{Float64,2}:
 0.5  0.0
 0.0  0.5

In [37]:
norm([2,0])

2.0

In [39]:
y = [3,6,2,4]

4-element Array{Int64,1}:
 3
 6
 2
 4